# Part I: Introduction

## 1. Generative vs. Discriminative Models
**<font color=orange>Generative model</font>:** 计算P(X, Y)\
以文本分类为例，生成模型的目标是求解：$\hat{y}=\underset{\tilde{y}}{argmax}P(X, \tilde{y})$

**<font color=orange>Discriminative model</font>:** 计算P(Y|X) \
以文本分类为例，判别模型的目标是求解：$\hat{y}=\underset{\tilde{y}}{argmax}P(\tilde{y}|X)$

